Imagenes

In [ ]:
# Importar las librerías necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Configuración de los parámetros
img_height, img_width = 128, 128  # Tamaño de las imágenes
batch_size = 32  # Tamaño de batch

# Directorio donde están las imágenes (cargar en Colab)
train_dir = 'ruta/del/dataset/entrenamiento'
val_dir = 'ruta/del/dataset/validacion'

# Generador de imágenes para aumentar y preprocesar los datos
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalización
    rotation_range=20,  # Aumentación de datos
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


In [ ]:
# Construcción de una CNN simple
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping]
)


In [ ]:
# Cargar ResNet50 preentrenada (sin la última capa) y congelar sus pesos
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Congelar la red base

# Añadir capas personalizadas encima de ResNet50
resnet_model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
history_resnet = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping]
)


In [ ]:
import matplotlib.pyplot as plt

# Función para graficar precisión y pérdida
def plot_history(history, title):
    plt.figure(figsize=(12, 5))

    # Precisión
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.title(f'{title} - Precisión')
    plt.legend()

    # Pérdida
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.title(f'{title} - Pérdida')
    plt.legend()

    plt.show()

# Graficamos para ambos modelos
plot_history(history_cnn, "CNN Simple")
plot_history(history_resnet, "ResNet50 Transfer Learning")


In [ ]:
# Desbloqueamos las últimas capas de ResNet50 para hacer ajuste fino
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Congelamos las capas iniciales
    layer.trainable = False

# Compilación del modelo con una tasa de aprendizaje baja
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Ajuste fino del modelo
history_resnet_finetune = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping]
)

**-------------------------------------------------------**

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Cargar dataset (asegúrate de subir el archivo 'disgrafia_data.csv' a tu Colab)
df = pd.read_csv('disgrafia_data.csv')

In [ ]:
# Visualizamos las primeras filas del dataset
print(df.head())

# Verificamos valores faltantes y los tratamos (ejemplo: eliminamos filas con valores nulos)
df.dropna(inplace=True)

# Codificamos las etiquetas si hay alguna variable categórica
label_encoder = LabelEncoder()
df['target'] = label_encoder.fit_transform(df['target'])  # 'target' es la columna objetivo

# Separación de características (X) y etiquetas (y)
X = df.drop(columns=['target'])  # Asegúrate de cambiar 'target' por el nombre de tu columna objetivo
y = df['target']

# División del dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Modelo 1: SVM (Máquinas de Vectores de Soporte)

In [ ]:
# Entrenamos un modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Evaluamos el modelo SVM
y_pred_svm = svm_model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

Modelo 2: Random Forest

In [ ]:
# Entrenamos un modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluamos el modelo Random Forest
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

In [ ]:
# Codificación one-hot para la salida (si el problema es de clasificación multicategoría)
num_classes = len(np.unique(y))
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# Construcción de la red neuronal
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compilación del modelo
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
history = nn_model.fit(X_train, y_train_categorical, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluación del modelo
nn_loss, nn_accuracy = nn_model.evaluate(X_test, y_test_categorical, verbose=0)
print("Neural Network Accuracy:", nn_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Gráfica de precisión del modelo
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Gráfica de pérdida del modelo
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()